In [1]:
import pandas as pd

In [2]:
file1 = pd.read_excel("data\Course_Section_Search_-_Central Term 1.xlsx", skiprows=1)
file2 = pd.read_excel("data\Course_Section_Search_-_Central Term 2 and Summer 2025.xlsx", skiprows=1)

In [3]:
file1_phys = file1.copy()
file2_phys = file2.copy()

In [4]:
phys_courses = pd.concat([file1_phys, file2_phys], ignore_index=True)
phys_courses.shape

(4091, 44)

In [5]:
phys_courses = phys_courses[phys_courses['Course Number'] < 500]

In [6]:
phys_courses.loc[:, 'Course Code'] = phys_courses['Course Subject'] + " " + phys_courses['Course Number'].astype(str)

In [7]:
phys_courses_final = phys_courses.drop_duplicates(subset=['Course Code'], keep='first')

In [8]:
phys_courses_final.shape

(658, 45)

### Extracting Requisites

In [9]:
import re

course_description = phys_courses_final[['Course Code', 'Description']].copy()
course_description.loc[:, 'reqs'] = course_description['Description'].str.extract(
    r'((prerequisite|corequisite)[\s\S]*)', 
    flags=re.IGNORECASE
)[0]
course_description.sample(5)

,Course Code,Description,reqs
3649,MICB_V 205,Foundations in data science course that incorp...,Prerequisite: DSCI 100.
3332,EOSC_V 333,"Analytical methods in geochemistry, major and ...",Prerequisite: EOSC 221.
2563,CAPS_V 426,An integrated study of the theory and experime...,"Prerequisite: One of CAPS 305, CAPS 301, NSCI ..."
2612,CHEM_V 205,Chemical kinetics and thermodynamics and spect...,Prerequisite: Either (a) SCIE 001 or (b) one o...
2558,CAPS_V 206,The principles of physiological regulation of ...,"Prerequisite: All of BIOL 112, CHEM 123, MATH ..."


In [10]:
pattern = r'\b[A-Z]{4}\s\d{3}\sis\srecommended\.'

# Remove the sentence if present
course_description['reqs'] = course_description['reqs'].str.replace(pattern, '', regex=True)
course_description.sample(3)

,Course Code,Description,reqs
1177,ENPH_V 259,"Basic experimental techniques in acquisition, ...","Prerequisite: One of APSC 176, ENGL 110, ENGL ..."
3267,ENVR_V 402,Instructor-guided collaboration between studen...,Prerequisite: ENVR 401.
1013,CPSC_V 349,Students will attend a series of research semi...,NaN


In [11]:
def extract_reqs(text, keyword):
    if isinstance(text, str):
        match = re.search(fr'{keyword}:.*?[.\]]', text)
        return match.group(0) if match else ''
    return ''

# Create 'prereqs' and 'coreqs' columns
course_description['prereqs'] = course_description['reqs'].apply(lambda x: extract_reqs(x, 'Prerequisite'))
course_description['coreqs'] = course_description['reqs'].apply(lambda x: extract_reqs(x, 'Corequisite'))

In [12]:
course_description['prereq_courses'] = course_description['prereqs'].apply(
    lambda x: re.findall(r'[A-Z]{4}\s*\d{3}', str(x)) if isinstance(x, str) else []
)
course_description['coreq_courses'] = course_description['coreqs'].apply(
    lambda x: re.findall(r'[A-Z]{4}\s*\d{3}', str(x)) if isinstance(x, str) else []
)
course_description.sample(5)

,Course Code,Description,reqs,prereqs,coreqs,prereq_courses,coreq_courses
2452,BIOL_V 340,Designing and conducting cell biology experime...,Prerequisite: BIOL 200 and at least 3rd-year s...,Prerequisite: BIOL 200 and at least 3rd-year s...,,"[BIOL 200, BIOL 180, BIOL 140, SCIE 001]",[]
3608,MATH_V 444,Current research topics in pure and applied ma...,"Prerequisite: One of MATH 220, MATH 226. and 6...","Prerequisite: One of MATH 220, MATH 226.",,"[MATH 220, MATH 226]",[]
2495,BIOL_V 412,Description and interpretation of present and ...,Prerequisite: BIOL 121,,,[],[]
656,CHEM_V 445,"Principles of experimental design, practice an...","Prerequisite: One of CHEM 345, CHEM 335. Stude...","Prerequisite: One of CHEM 345, CHEM 335.",,"[CHEM 345, CHEM 335]",[]
1705,MRNE_V 480,Instruction in the critical analysis of publis...,Prerequisite: Completion of second year in a b...,Prerequisite: Completion of second year in a b...,,[],[]


In [13]:
course_description.drop(columns=['prereqs', 'coreqs', 'reqs'], inplace=True)

In [14]:
course_description.sample()

,Course Code,Description,prereq_courses,coreq_courses
3887,PHYS_V 310,Fundamental principles and applications of dat...,"[MATH 152, MATH 221, MATH 223, MATH 200, MATH ...",[]


In [15]:
def standardize_courses(list):
    standardized_list = []
    for course in list:
        # Use regex to find courses with no space before the number
        standardized_course = re.sub(r'([A-Z]+)(\d+)', r'\1 \2', course)
        standardized_list.append(standardized_course)
    return standardized_list

# Apply the function to the prereq_courses column
course_description['prereq_courses'] = course_description['prereq_courses'].apply(standardize_courses)
course_description['coreq_courses'] = course_description['coreq_courses'].apply(standardize_courses)

In [16]:
course_description['Course Code'] = course_description['Course Code'].str.replace('_V', '', regex=False)

In [17]:
course_description.sample()

,Course Code,Description,prereq_courses,coreq_courses
1712,NSCI 302,Physiological mechanisms of neurological and p...,[NSCI 201],[]


In [20]:
themes = pd.read_csv('data/course_themes.csv')
themes.head(1)

,Department,Course Code,Human Health,Data and Computation,AI,Clean Energy and Materials,Climate,Science and Society,Notes
0,CAPS,ANAT_V 392,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
themes.drop(columns=['Department', 'Notes'], inplace=True)
themes.head(1)

,Course Code,Human Health,Data and Computation,AI,Clean Energy and Materials,Climate,Science and Society
0,ANAT_V 392,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
themes.iloc[:, 1:] = themes.iloc[:, 1:].notna() 
themes.head(1)

C:\Users\mahen\AppData\Local\Temp\ipykernel_9332\2876175342.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  themes.iloc[:, 1:] = themes.iloc[:, 1:].notna()


,Course Code,Human Health,Data and Computation,AI,Clean Energy and Materials,Climate,Science and Society
0,ANAT_V 392,False,False,False,False,False,False


In [23]:
themes['Course Code'] = themes['Course Code'].str.replace('_V', '', regex=False)
themes.sample()

,Course Code,Human Health,Data and Computation,AI,Clean Energy and Materials,Climate,Science and Society
117,BIOL 439,False,False,False,False,False,False


In [24]:
themes['themes'] = themes.iloc[:, 1:].apply(lambda x: list(themes.columns[1:][x]), axis=1)

,Course Code,Human Health,Data and Computation,AI,Clean Energy and Materials,Climate,Science and Society,themes
391,MICB 301,False,False,False,False,False,False,[]
249,ENPH 352,False,False,False,False,False,False,[]
102,BIOL 417,False,False,False,False,False,False,[]
57,BIOL 310,False,False,False,False,False,False,[]
604,CPSC 490,False,False,False,False,False,False,[]
357,MATH 307,False,False,False,False,False,False,[]
425,PCTH 201,False,False,False,False,False,False,[]
179,CHEM 319,False,False,False,False,False,False,[]
64,BIOL 325,False,False,False,False,False,False,[]
138,BIOL 465,False,False,False,False,False,False,[]


In [26]:
themes = themes[['Course Code', 'themes']]
themes.sample()

,Course Code,themes
348,MATH 223,[]


In [27]:
courses_with_themes = course_description.merge(themes, on='Course Code', how='left')

In [28]:
courses_with_themes.sample(10)

,Course Code,Description,prereq_courses,coreq_courses,themes
69,BIOL 399,Work experience in an industrial research sett...,[BIOL 398],[],[]
572,CHEM 410,"Electromagnetic, optical, thermal and mechanic...","[CHEM 304, PHYS 203, CHEM 312, PHYS 304]",[],[]
52,BIOL 325,Comparative aspects of the functional design o...,[],[],[]
210,ENVR 300,Environmental research. Students investigate ...,[],[],[Climate]
481,BIOL 310,Animal behavior from an ecological and evoluti...,[BIOL 121],[],[]
229,EOSC 310,"The Earth as a planet: its composition, intern...",[],[],[Climate]
75,BIOL 421,Biology and physiology of selected plant-micro...,"[BIOL 200, BIOC 202, BIOC 203, BIOL 201, BIOL ...",[],[]
59,BIOL 342,Field-based and laboratory-based investigation...,"[BIOL 121, BIOL 180, BIOL 140, SCIE 001]",[],[]
395,PHYS 309,Selected experiments in electromagnetism and e...,"[PHYS 209, PHYS 229, ENPH 259]",[],[]
136,CHEM 341,Importance of chemistry in society. Detailed c...,"[CHEM 203, CHEM 223, CHEM 233, CHEM 260]",[],[]


In [30]:
# Use in case of extracting all prereqs

import json

phys_courses_json = []

for _, row in courses_with_themes.iterrows():
    course_entry = {
        "course_code": row['Course Code'],
        "description": row['Description'],
        "prerequisites": row['prereq_courses'],  # Directly using the list from the CSV
        "corequisites": row['coreq_courses'],  # You can include other columns as needed
        "themes": row['themes']
    }
    phys_courses_json.append(course_entry)

# Convert to JSON string (optional, for saving or viewing)
courses_json_str = json.dumps(phys_courses_json, indent=4)

# Save the JSON to a file
with open('data/all_courses.json', 'w') as json_file:
    json_file.write(courses_json_str)

In [33]:
# Load the JSON data
with open('data/all_courses.json', 'r') as file:
    data = json.load(file)

# Extract all valid course codes into a set for fast lookup
valid_course_codes = {course['course_code'] for course in data}

# Filter the prerequisites for each course
for course in data:
    # Keep only those prerequisites that are in the valid course codes
    course['prerequisites'] = [prereq for prereq in course['prerequisites'] if prereq in valid_course_codes]

for course in data:
    course['corequisites'] = [coreq for coreq in course['corequisites'] if coreq in valid_course_codes]

# Save the modified data back to the JSON file
with open('data/all_courses.json', 'w') as file:
    json.dump(data, file, indent=4)

print("Prerequisites filtered successfully!")

Prerequisites filtered successfully!
